In [49]:
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import http.client
import requests
from urllib.request import urlopen
import json
import pandas as pd


In [2]:
user = "postgres"
password = "Ww4756"
host = "localhost"
port = 5432
database = "casestudy"
conn = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(conn)

In [39]:
conn = http.client.HTTPSConnection("api.collectapi.com")

headers = {
    'content-type': "application/json",
    'authorization': "apikey 2MCBVYZ5Dy8s4XFXw6CYXN:37926RomyR3skJsPe7Kr7D"
    }

conn.request("GET", "/health/dutyPharmacy?ilce=%C3%87ankaya&il=Ankara", headers=headers)

res = conn.getresponse()
data = res.read()

In [40]:
my_json = data.decode('utf8').replace("'", '"')
data2= json.loads(my_json)
s = json.dumps(data2, indent=4, sort_keys=True)


In [77]:
df=pd.DataFrame.from_dict(data2["result"])

In [78]:
df[['lat', 'lng']] = df['loc'].str.split(',', expand=True)
df.drop(columns=['loc'], inplace=True)


In [79]:
pharmacy= gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.lng, df.lat))
pharmacy=pharmacy.drop(columns=['lng','lat'])
pharmacy.geometry.crs = {'init':'epsg:4326'}


/Users/eyuptunahanunal/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [80]:
pharmacy.to_postgis(name="pharmacy_on_duty", con=engine, schema="public")